# **DEEP LEARNING COURSE PROJECT (IMAGE CLASSIFICATION)**

### **NAME:** ANISH SRIRAM B S
### **SCHOOL:** SCDS
### **EMAIL:** anishsriram.b-26@scds.saiuniversity.edu.in

## TRANSFER LEARNING - RESNET101V2 MODEL

### LOADING THE DATASET

### Details:

*   Total Number of Images in the Dataset: 1661
*   Number of Images present per class:
    *   Number of Images in Motorbikes: 500
    *   Number of Images in Airplanes: 500
    *   Number of Images in Schooner: 63
*   Total Number of Images used for Training and Testing: 1063
    *   Number of Images used for Training: 797
    *   Number of Images used for Testing: 266












In [1]:
# Importing necessary modules
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import glob
import PIL
from PIL import Image
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from collections import Counter
from numpy import save
from numpy import load

In [2]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [3]:
# Loading the dataset which is already processed
X_train_std = load('/content/drive/MyDrive/Deep Learning Project/Dataset/X_train_std.npy')
X_test_std = load('/content/drive/MyDrive/Deep Learning Project/Dataset/X_test_std.npy')
y_train = load('/content/drive/MyDrive/Deep Learning Project/Dataset/y_train.npy')
y_test = load('/content/drive/MyDrive/Deep Learning Project/Dataset/y_test.npy')

print("X_train_std_shape:",X_train_std.shape)
print("X_test_std_shape:",X_test_std.shape)
print("y_train_shape:",y_train.shape)
print("y_test_shape:",y_test.shape)

X_train_std_shape: (797, 299, 299, 3)
X_test_std_shape: (266, 299, 299, 3)
y_train_shape: (797,)
y_test_shape: (266,)


### TRANSFER LEARNING

In [4]:
# Loading the feature extractor part of the ResNet101V2 model
base_model = keras.applications.ResNet101V2(weights='imagenet',include_top=False)

# Setting the weights of the feature extractor part as non trainable
for layer in base_model.layers:
  layer.trainable = False

# Defining the classifier part
global_avg_pooling = keras.layers.GlobalAveragePooling2D()(base_model.output) # Performing Global Average Pooling
x_batchnorm = keras.layers.BatchNormalization()(global_avg_pooling) # Performing Batch Normalization
x_dropout = keras.layers.Dropout(0.35)(x_batchnorm) # Performing Dropout with 35% drop rate
output_ = keras.layers.Dense(units=3,activation="softmax")(x_dropout) # Output layer

model_2_TL = keras.models.Model(inputs=[base_model.input],outputs=[output_])



171317808/171317808 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [5]:
# Compiling the model
model_2_TL.compile(loss="sparse_categorical_crossentropy",optimizer="adam",metrics=["accuracy"])

# Saving the best model using Callbacks
callbacks_2_TL = [keras.callbacks.ModelCheckpoint("Best_model_2_TL.weights.h5",monitor='val_accuracy',save_weights_only=True,save_best_only=True)]

# Training the model
train_model_2_TL = model_2_TL.fit(x=X_train_std,y=y_train,epochs=10,validation_split=0.1,batch_size=16,callbacks=callbacks_2_TL)


Epoch 1/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 44s 517ms/step - accuracy: 0.6901 - loss: 0.7564 - val_accuracy: 0.9125 - val_loss: 0.2536
Epoch 2/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 14s 130ms/step - accuracy: 0.9243 - loss: 0.1826 - val_accuracy: 0.9125 - val_loss: 0.2013
Epoch 3/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 11s 152ms/step - accuracy: 0.9509 - loss: 0.1118 - val_accuracy: 0.9250 - val_loss: 0.1699
Epoch 4/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 6s 130ms/step - accuracy: 0.9771 - loss: 0.0636 - val_accuracy: 0.9125 - val_loss: 0.1633
Epoch 5/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 6s 128ms/step - accuracy: 0.9592 - loss: 0.1136 - val_accuracy: 0.9250 - val_loss: 0.1613
Epoch 6/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 7s 158ms/step - accuracy: 0.9737 - loss: 0.0612 - val_accuracy: 0.9625 - val_loss: 0.0957
Epoch 7/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 9s 131ms/step - accuracy: 0.9766 - loss: 0.0592 - val_accuracy: 0.9375 - val_loss: 0.1061
Epoch 8/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 13s 192ms/step - accuracy: 0.9721 - loss: 0.0764 - val_accuracy

### SAVING THE MODEL

In [6]:
# Saving the best model obtained
model_2_TL.save('/content/drive/MyDrive/Deep Learning Project/Model2/Best_model_2_TL.keras')